In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
import seaborn
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
from pycaret.classification import *
from pycaret.classification import evaluate_model
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import selenium
from selenium import webdriver
import random
from selenium.webdriver.common.by import By
from sklearn.preprocessing import MinMaxScaler

C:\Users\idotz\anaconda3\envs\p37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\idotz\anaconda3\envs\p37\lib\site-packages\numpy\.libs\libopenblas.4SP5SUA7CBGXUEOC35YP2ASOICYYEQZZ.gfortran-win_amd64.dll
C:\Users\idotz\anaconda3\envs\p37\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [16]:
df = pd.read_csv('data_0905.csv', index_col = [0])

In [19]:
df[df['odds_predict'] != df['final_score']].head(20)

,Expected goals (xG),Total shots,Big chances,Big chances missed,Accurate passes,Accurate passes percentage,Fouls committed,Offsides,Corners,Shots off target,...,Aerial duels won percentage,Successful dribbles,Successful dribbles percentage,score,posession,0,1,2,final_score,odds_predict
3,0.37,4.0,1.0,1.0,56.0,0.04,-1.0,-1.0,-1.0,3.0,...,-0.36,3.0,0.04,-1.0,12.0,3.500000,6.500000,1.600000,1,2
10,-1.56,-10.0,-2.0,-2.0,-102.0,-0.16,3.0,2.0,-6.0,-3.0,...,0.12,3.0,0.22,0.0,-26.0,3.250000,6.500000,1.615385,1,2
11,0.15,3.0,0.0,0.0,104.0,0.07,-4.0,1.0,1.0,0.0,...,0.00,-1.0,0.17,0.0,20.0,2.375000,2.250000,5.000000,2,1
12,-0.88,-4.0,1.0,1.0,-100.0,-0.10,-3.0,1.0,-4.0,1.0,...,0.00,0.0,0.38,1.0,-30.0,3.500000,1.531667,7.500000,0,1
17,-0.39,-6.0,-1.0,-1.0,-21.0,-0.04,-5.0,2.0,-6.0,-3.0,...,0.34,-5.0,-0.31,0.0,-6.0,2.477273,3.112500,2.800000,2,0
20,-0.26,-4.0,0.0,-1.0,16.0,-0.02,2.0,-1.0,0.0,0.0,...,0.00,0.0,0.11,0.0,8.0,2.550000,2.600000,3.275000,2,0
23,0.73,2.0,2.0,1.0,6.0,0.06,3.0,2.0,5.0,0.0,...,0.14,0.0,-0.60,0.0,2.0,2.500000,2.750000,3.075000,2,0
25,-0.30,-8.0,0.0,0.0,-58.0,-0.09,4.0,0.0,-2.0,-5.0,...,-0.28,0.0,0.00,0.0,-12.0,2.612500,2.860577,2.875000,2,0
26,0.03,-1.0,2.0,1.0,-202.0,-0.18,-1.0,0.0,-1.0,0.0,...,0.12,-7.0,-0.16,1.0,-40.0,3.400000,1.666667,5.500000,0,1
28,-0.24,1.0,-1.0,-1.0,-33.0,0.00,-5.0,3.0,-3.0,2.0,...,-0.34,-2.0,0.00,0.0,-10.0,2.250000,6.750000,2.100000,0,2


In [69]:
fot_df = pd.read_csv('current_stats.csv', index_col = [0])

In [311]:
def winner(x):
    if x == 0:
        return 0
    elif x > 0:
        return 1
    else:
        return 2

In [115]:
df = pd.read_csv('og_half.csv', index_col = [0])
df = df[half_cols]

In [642]:
df = pd.read_csv('fulltime.csv', index_col = [0])
df = df[df['Expected goals (xG)'].notna()]
df = df.drop(['url', 'xG penalty'], axis = 1)
# df['league'] = pd.factorize(df['league'])[0]
df['target'] = df['score'].apply(winner)

In [22]:
df

,Corners,Fouls Committed,Goals,Offsides,Pass Completion,Possesion,Red Cards,Shot Attempts,Shots Blocked,Total Passes,Total Shots on Target,Total xG,Yellow Cards,target
0,-4.0,-1.0,-1.0,0.0,0.149,0.248,0.0,-6.0,-3.0,115.0,-1.0,-0.806711,0.0,2
1,0.0,0.0,0.0,2.0,0.157,0.316,0.0,-2.0,-4.0,178.0,0.0,-0.323895,0.0,1
2,1.0,0.0,0.0,0.0,-0.097,-0.064,0.0,3.0,2.0,-35.0,3.0,-0.247529,-1.0,2
3,0.0,1.0,1.0,-1.0,0.023,-0.008,0.0,0.0,2.0,-5.0,-2.0,0.722193,1.0,1
4,0.0,0.0,-1.0,0.0,-0.114,-0.360,0.0,-1.0,1.0,-198.0,-2.0,-0.525874,2.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677,6.0,0.0,0.0,0.0,0.134,0.256,0.0,12.0,3.0,120.0,4.0,0.593374,-1.0,2
678,-4.0,3.0,-1.0,1.0,-0.180,-0.320,0.0,-13.0,-3.0,-153.0,-6.0,-1.904855,-1.0,2
679,-6.0,1.0,1.0,0.0,-0.149,-0.292,0.0,-5.0,-2.0,-121.0,-3.0,-0.460209,1.0,1
680,-1.0,0.0,2.0,0.0,0.144,0.330,0.0,5.0,1.0,196.0,0.0,1.188723,0.0,1


In [23]:
# df = df[scraped_cols]
scaler = MinMaxScaler()
df[df.columns[:-1]] = scaler.fit_transform(df[df.columns[:-1]])

In [114]:
half_cols = ['Corners', 'Goals', 'Offsides','Pass Completion', 'Possesion', 'Red Cards', 'Shot Attempts',
       'Shots Blocked', 'Total Passes', 'Total Shots on Target', 'Total xG', 'Yellow Cards', 'target']

In [67]:
scraped_cols = ['Corners', 'Fouls committed', 'score', 'Offsides', 'Accurate passes percentage',
       'posession','Red cards','Total shots', 'Blocks','Passes', 'Shots on target','Expected goals (xG)',
       'Yellow cards','url']

In [5]:
train, test = train_test_split(df, test_size=0.2)

In [10]:
print(train.shape)
print(test.shape)

(545, 14)
(137, 14)


In [22]:
grid = setup(data=df, target='final_score', feature_selection = True,  ignore_features = ['0','1','2'], remove_multicollinearity = True, feature_selection_threshold = 0.8)
#             feature_ratio = True, feature_interaction = True)

,Description,Value
0,session_id,1349
1,Target,final_score
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(196, 48)"
5,Missing Values,True
6,Numeric Features,43
7,Categorical Features,1
8,Ordinal Features,False
9,High Cardinality Features,False


In [23]:
best = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.6654,0.8247,0.6344,0.6843,0.6486,0.4786,0.4985,0.0900
rf,Random Forest Classifier,0.6434,0.8069,0.5739,0.6055,0.6038,0.4268,0.4588,0.1960
et,Extra Trees Classifier,0.6357,0.8180,0.5644,0.5880,0.5933,0.4153,0.4454,0.1780
xgboost,Extreme Gradient Boosting,0.6280,0.7812,0.5783,0.6128,0.5977,0.4156,0.4395,0.0420
lr,Logistic Regression,0.6209,0.8058,0.5883,0.6566,0.6100,0.4158,0.4336,0.0660
ridge,Ridge Classifier,0.6071,0.0000,0.5528,0.6111,0.5883,0.3874,0.4033,0.0060
lda,Linear Discriminant Analysis,0.6071,0.8022,0.5694,0.6183,0.5929,0.3965,0.4163,0.0060
nb,Naive Bayes,0.6066,0.8089,0.5622,0.6301,0.6018,0.3894,0.3984,0.0050
ada,Ada Boost Classifier,0.5989,0.7636,0.5756,0.6375,0.5886,0.3935,0.4112,0.0250
dt,Decision Tree Classifier,0.5912,0.6867,0.5494,0.6126,0.5820,0.3624,0.3763,0.0050


In [24]:
tuned = tune_model(best)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5714,0.7601,0.4889,0.5214,0.5315,0.3171,0.3375
1,0.8571,0.9407,0.8333,0.8690,0.8571,0.7795,0.7857
2,0.4286,0.6271,0.3556,0.3878,0.4066,0.0894,0.0902
3,0.7143,0.7851,0.6111,0.5655,0.6308,0.5254,0.5637
4,0.6429,0.8418,0.5778,0.7095,0.6071,0.4068,0.4611
5,0.5000,0.6587,0.4667,0.3571,0.4167,0.2222,0.2481
6,0.7857,0.9706,0.7667,0.8661,0.7827,0.6719,0.7127
7,0.9231,0.9582,0.8889,0.9341,0.9183,0.8762,0.8853
8,0.5385,0.8444,0.5000,0.6205,0.5692,0.3036,0.3119


In [25]:
evaluate_model(tuned)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [26]:
preds = predict_model(tuned, raw_score = True)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Gradient Boosting Classifier,0.5085,0.7107,0.4556,0.4568,0.4637,0.2226,0.2330


In [15]:
preds

,0,Accurate long balls percentage,xG open play,Shots outside box,Corners,Own half,odds_precict_1,Accurate crosses percentage,Expected goals (xG),Aerial duels won,Duels won,Accurate long balls,1,Total shots,2,final_score,Label
0,3.300000,0.09,0.250000,0.0,5.0,-23.0,0.0,-0.100000,0.82,7.0,-6.0,5.0,6.000000,6.0,1.666667,0,0
1,2.100000,0.23,0.010000,0.0,-1.0,77.0,0.0,-0.570000,-0.16,-7.0,2.0,13.0,2.200000,-4.0,7.000000,1,2
2,4.333333,0.16,-0.270000,-3.0,-1.0,53.0,1.0,-0.200000,-0.04,1.0,5.0,5.0,1.333333,-2.0,11.000000,1,1
3,3.000000,0.24,0.780000,5.0,1.0,57.0,1.0,0.400000,0.85,-2.0,-6.0,0.0,1.571429,11.0,9.375000,2,1
4,2.612500,0.06,-0.210000,-4.0,-2.0,-25.0,0.0,-0.360000,-0.30,-3.0,-6.0,4.0,2.860577,-8.0,2.875000,2,2
5,5.500000,0.08,0.011825,1.0,1.0,-16.0,1.0,0.170000,0.00,0.0,8.0,0.0,1.250000,8.0,13.000000,1,1
6,2.500000,0.05,0.070000,3.0,0.0,-16.0,0.0,0.070000,0.02,1.0,-1.0,1.0,4.550000,1.0,2.150000,2,2
7,3.500000,-0.01,0.370000,2.0,-1.0,-14.0,0.0,0.400000,0.37,-6.0,-5.0,-4.0,6.500000,4.0,1.600000,1,2
8,2.200000,-0.05,-0.140000,1.0,1.0,-5.0,0.0,0.080000,-0.06,3.0,6.0,-8.0,3.800000,1.0,2.800000,1,2
9,4.750000,0.18,-0.010000,7.0,5.0,4.0,1.0,0.250000,0.37,0.0,-8.0,-5.0,1.250000,14.0,15.000000,0,1
